In [1]:
from astropy.io import fits
import os
from pathlib import Path
import random
import pandas as pd
from astropy.visualization import make_lupton_rgb
from astropy.io import fits
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import ascii as astro_ascii
from astropy.table import Table
import subprocess

In [ ]:
DES_DIR = Path(
    "/nfs/turbo/lsa-regier/scratch/gapatron/desdr-server.ncsa.illinois.edu/despublic/dr2_tiles/"
)
DES_SUBDIR = os.listdir(DES_DIR)[0]
DES_SUBDIR

In [ ]:
main_path = DES_DIR / Path(DES_SUBDIR) / Path(f"{DES_SUBDIR}_dr2_main.fits")
flux_path = DES_DIR / Path(DES_SUBDIR) / Path(f"{DES_SUBDIR}_dr2_flux.fits")
main_data = fits.getdata(main_path)
main_df = pd.DataFrame(main_data)
flux_data = fits.getdata(flux_path)
flux_df = pd.DataFrame(flux_data)
full_df = pd.merge(
    main_df, flux_df, left_on="COADD_OBJECT_ID", right_on="COADD_OBJECT_ID", how="left"
)
full_df

In [4]:
fluxes = np.array(
            full_df[
                [
                    "FLUX_AUTO_G_x",
                    "FLUX_AUTO_R_x",
                    "FLUX_AUTO_I_x",
                    "FLUX_AUTO_Z_x",
                ]
            ]
        )

fluxes *= (fluxes > 0)

hlrs = 0.263 * np.array(full_df["FLUX_RADIUS_R"])
hlrs = 1e-4 + hlrs * (hlrs > 0)
a = np.array(full_df["A_IMAGE"])
b = np.array(full_df["B_IMAGE"])
g = (a - b) / (a + b)
angle = np.arctan(b / a)
g1 = g * np.cos(angle)
g2 = g * np.sin(angle)


In [5]:
mock_catalog = pd.DataFrame()
mock_catalog["RA"] = np.array(full_df["ALPHAWIN_J2000_x"])
mock_catalog["DEC"] = np.array(full_df["DEC_x"])
mock_catalog["X"] = np.array(full_df["XWIN_IMAGE_R"])
mock_catalog["Y"] = np.array(full_df["YWIN_IMAGE_R"])
mock_catalog["MEM"] = 0
mock_catalog["FLUX_R"] = fluxes[:, 1]
mock_catalog["FLUX_G"] = fluxes[:, 0]
mock_catalog["FLUX_I"] = fluxes[:, 2]
mock_catalog["FLUX_Z"] = fluxes[:, 3]
mock_catalog["HLR"] = hlrs
mock_catalog["FRACDEV"] = 0
mock_catalog["G1"] = g1
mock_catalog["G2"] = g2
mock_catalog["Z"] = 0
mock_catalog["SOURCE_TYPE"] = 0

In [7]:
DATA_PATH = "/home/kapnadak/bliss/case_studies/galaxy_clustering/test_des_data"
CATALOG_PATH = os.path.join(DATA_PATH, "catalogs")
if not os.path.exists(CATALOG_PATH):
    os.makedirs(CATALOG_PATH)

file_ext = "test_des_000.dat"
file_name = f"{CATALOG_PATH}/{file_ext}"
catalog_table = Table.from_pandas(mock_catalog)
astro_ascii.write(catalog_table, file_name, format="no_header", overwrite=True)

In [ ]:
image_size = 10000
nfiles = 1
data_dir = DATA_PATH
input_dir = f"{data_dir}/catalogs"
output_dir = f"{data_dir}/images"
output_ext = "test_des_000_g.fits"
psf_dir = "/nfs/turbo/lsa-regier/scratch/gapatron/psf-models/dr2_tiles_reformatted/gband"
psf_file = "galsim_des_000_g.psf"
args = []
GALSIM_PATH = "/home/kapnadak/bliss/case_studies/galaxy_clustering/data_generation/custom-single-image-galsim.yaml"
args.append("galsim")
args.append(f"{GALSIM_PATH}")
args.append(f"variables.image_size={image_size}")
args.append(f"variables.nfiles={nfiles}")
args.append(f"variables.input_dir={input_dir}")
args.append(f"variables.input_file={file_ext}")
args.append(f"variables.output_dir={output_dir}")
args.append(f"variables.output_file={output_ext}")
args.append(f"variables.psf_dir={psf_dir}")
args.append(f"variables.psf_file={psf_file}")
subprocess.run(args, shell=False, check=False)